<a href="https://colab.research.google.com/github/rhaymisonbetini/DLR_IFS/blob/main/IA_TRADER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas-datareader
!pip install -q yfinance

In [7]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader
import yfinance as yf
from tqdm import tqdm_notebook, tqdm
from collections import deque

In [3]:
class Trader():
    def __init__(self, state_size, action_space = 3, model_name = "Trader"):
        self.state_size = state_size
        self.action_space = action_space
        self.model_name = model_name
        self.memory = deque(maxlen = 1000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_final = 0.01
        self.epsilon_decay = 0.995
        self.model = self.model_builder

    def model_builder(self):
      model = tf.keras.models.Sequential()
      model.add(tf.keras.layers.Dense(units = 32, activation = "relu", input_dim = self.state_size))
      model.add(tf.keras.layers.Dense(units = 64, activation = "relu"))
      model.add(tf.keras.layers.Dense(units = 128, activation = "relu"))
      model.add(tf.keras.layers.Dense(units=self.action_space, activation = "linear"))
      model.compile(loss = "mse", optimizer = tf.keras.optimizers.Adam(lr = 0.001))
      return model

    def trade(self, state):
        if random.random() <= self.epsilon:
            return random.randrange(self.action_space)
        actions = self.model.predict(state)
        return np.argmax(actions[0])

    def batch_train(self, batch_size):
        batch = []
        for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
            batch.append(self.memory[i])
        for state, action, reward, next_state, done in batch:
            if not done:
                reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            target = self.model.predict(state)
            target[0][action] = reward
            self.model.fit(state, target, epochs = 1, verbose = 0)
        if self.epsilon > self.epsilon_final:
            self.epsilon *= self.epsilon_decay

In [4]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [5]:
def stocks_price_format(n):
    if n < 0:
        return "- $ {0:2f}".format(abs(n))
    else:
        return "$ {0:2f}".format(abs(n))

In [8]:
def dataset_loader(stock_name):
  dataset = yf.download(stock_name, start='2016-06-02')
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  close = dataset['Close']
  return close

[*********************100%%**********************]  1 of 1 completed


In [11]:
def state_creator(data, timestep, window_size):
  starting_id = timestep - window_size + 1
  if starting_id >= 0:
    windowed_data = data[starting_id: timestep + 1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep + 1])
  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i + 1] - windowed_data[i]))
  return np.array([state])